In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# Chroma DB에서 문서 삭제
Chroma().delete_collection()

# HuggingfaceEmbedding 함수로 Open source 임베딩 모델 로드
ko_embedding = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
)

# PDF 파일 로드 및 분할
loader = PyPDFLoader("data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

# 청크 크기 및 중첩 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
docs = text_splitter.split_documents(pages)

# ChromaDB에 청크들을 벡터 임베딩으로 저장
db = Chroma.from_documents(docs, ko_embedding)

/opt/anaconda3/envs/RAG_CODE/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_ollama import ChatOllama
from langchain.retrievers.multi_query import MultiQueryRetriever

# question으로 질문 저장
question = "국회의원의 의무는 무엇이 있나요?"

# 여러 버전의 질문으로 변환하는 역할을 맡을 LLM 선언
llm = ChatOllama(
    model = "basic_kanana",
    temperature = 0
)

# MultiQueryRetriever에 벡터DB 기반 Retriever와 LLM 선언
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever = db.as_retriever(),
    llm = llm
)

# 여러 버전의 문장 생성 결과를 확인하기 위한 로깅 과정
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# 여러 버전 질문 생성 결과와 유사 청크 검색 개수 출력
unique_docs = retriever_from_llm.invoke(input=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 국회의원이 지켜야 할 주요 책임과 의무는 무엇인가요?', '2. 국회의원의 역할과 그들이 수행해야 하는 구체적인 임무는 무엇인지 알고 싶습니다.', '3. 국회의원의 권한과 책임을 설명해 주세요.']


7